In [3]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash, dcc, html, Input, Output, State

# Function to generate harmonic with noise
def harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise, time):
    harmonic = amplitude * np.sin(2 * np.pi * frequency * time + phase)
    noise = np.random.normal(noise_mean, np.sqrt(noise_covariance), size=len(time))
    if show_noise:
        return harmonic + noise, noise
    return harmonic, None

# Custom filter function
def custom_filter(signal, window_size):
    if window_size < 1:
        raise ValueError("Window size must be >= 1")

    kernel = np.ones(window_size) / window_size
    filtered_signal = np.convolve(signal, kernel, mode='same')
    return filtered_signal

# Initial parameters
initial_params = {
    "amplitude": 1.0,
    "frequency": 1.0,
    "phase": 0.0,
    "noise_mean": 0.0,
    "noise_covariance": 0.1,
    "show_noise": True,
    "filter_window_size": 5
}

# Time range
time = np.linspace(0, 2, 1000)

# Create Dash app
app = Dash(__name__)

# User interface
app.layout = html.Div([
    html.H1("Harmonic with Noise", style={"textAlign": "center", "color": "white"}),

    dcc.Graph(id="harmonic-plot"),

    html.Div([
        html.Div([
            html.Label(id="amplitude-label", children=f"Amplitude: {initial_params['amplitude']:.1f}", style={"color": "white"}),
            dcc.Slider(id="amplitude-slider", min=0.1, max=5.0, step=0.1, value=initial_params["amplitude"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="frequency-label", children=f"Frequency: {initial_params['frequency']:.1f}", style={"color": "white"}),
            dcc.Slider(id="frequency-slider", min=0.1, max=5.0, step=0.1, value=initial_params["frequency"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="phase-label", children=f"Phase Shift: {initial_params['phase']:.1f}", style={"color": "white"}),
            dcc.Slider(id="phase-slider", min=0.0, max=2 * np.pi, step=0.1, value=initial_params["phase"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="noise-mean-label", children=f"Noise Mean: {initial_params['noise_mean']:.1f}", style={"color": "white"}),
            dcc.Slider(id="noise-mean-slider", min=-1.0, max=1.0, step=0.1, value=initial_params["noise_mean"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="noise-covariance-label", children=f"Noise Covariance: {initial_params['noise_covariance']:.2f}", style={"color": "white"}),
            dcc.Slider(id="noise-covariance-slider", min=0.01, max=1.0, step=0.01, value=initial_params["noise_covariance"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="filter-window-label", children=f"Filter Window Size: {initial_params['filter_window_size']}", style={"color": "white"}),
            dcc.Slider(id="filter-window-slider", min=1, max=50, step=1, value=initial_params["filter_window_size"]),
        ], style={"margin-bottom": "20px"}),

        html.Label("Show Noise", style={"color": "white"}),
        dcc.Checklist(id="show-noise-checkbox", options=[{"label": "Noise", "value": "show_noise"}],
                      value=["show_noise"] if initial_params["show_noise"] else [], style={"color": "white"}),

        html.Label("Select Plot", style={"color": "white"}),
        dcc.Dropdown(
            id="plot-dropdown",
            options=[
                {"label": "Harmonic Only", "value": "harmonic"},
                {"label": "Noise Only", "value": "noise"},
                {"label": "Both", "value": "both"},
                {"label": "Filtered Harmonic", "value": "filtered"},
            ],
            value="both",
        ),

        html.Button("Reset", id="reset-button", n_clicks=0, style={"margin-top": "20px", "background-color": "#4CAF50", "color": "white", "border": "none", "padding": "10px 20px", "cursor": "pointer"}),
    ], style={"width": "50%", "margin": "auto"})
], style={"background-color": "#2c3e50", "padding": "20px"})

# Update the plot
@app.callback(
    Output("harmonic-plot", "figure"),
    Input("amplitude-slider", "value"),
    Input("frequency-slider", "value"),
    Input("phase-slider", "value"),
    Input("noise-mean-slider", "value"),
    Input("noise-covariance-slider", "value"),
    Input("filter-window-slider", "value"),
    Input("show-noise-checkbox", "value"),
    Input("plot-dropdown", "value"),
    State("harmonic-plot", "figure")
)
def update_plot(amplitude, frequency, phase, noise_mean, noise_covariance, filter_window, show_noise, plot_selection, current_figure):
    show_noise_flag = "show_noise" in show_noise
    harmonic, noise = harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise_flag, time)

    if plot_selection == "filtered":
        harmonic = custom_filter(harmonic, filter_window)

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Harmonic", "Noise"))

    # Display graphs based on dropdown selection
    if plot_selection in ["harmonic", "both", "filtered"]:
        fig.add_trace(go.Scatter(x=time, y=harmonic, mode="lines", name="Harmonic", line=dict(color="cyan")), row=1, col=1)

    if plot_selection in ["noise", "both"] and noise is not None:
        fig.add_trace(go.Scatter(x=time, y=noise, mode="lines", name="Noise", line=dict(color="magenta", dash="dot")), row=2, col=1)

    fig.update_layout(
        height=600,
        title="Harmonic with Noise",
        title_font=dict(color="white"),
        plot_bgcolor="#34495e",
        paper_bgcolor="#2c3e50",
        font=dict(color="white")
    )

    return fig

# Update slider labels
@app.callback(
    [Output("amplitude-label", "children"),
     Output("frequency-label", "children"),
     Output("phase-label", "children"),
     Output("noise-mean-label", "children"),
     Output("noise-covariance-label", "children"),
     Output("filter-window-label", "children")],
    [Input("amplitude-slider", "value"),
     Input("frequency-slider", "value"),
     Input("phase-slider", "value"),
     Input("noise-mean-slider", "value"),
     Input("noise-covariance-slider", "value"),
     Input("filter-window-slider", "value")]
)
def update_labels(amplitude, frequency, phase, noise_mean, noise_covariance, filter_window):
    return (
        f"Amplitude: {amplitude:.1f}",
        f"Frequency: {frequency:.1f}",
        f"Phase Shift: {phase:.1f}",
        f"Noise Mean: {noise_mean:.1f}",
        f"Noise Covariance: {noise_covariance:.2f}",
        f"Filter Window Size: {filter_window}"
    )

# Reset parameters
@app.callback(
    [Output("amplitude-slider", "value"),
     Output("frequency-slider", "value"),
     Output("phase-slider", "value"),
     Output("noise-mean-slider", "value"),
     Output("noise-covariance-slider", "value"),
     Output("filter-window-slider", "value"),
     Output("show-noise-checkbox", "value"),
     Output("plot-dropdown", "value")],
    Input("reset-button", "n_clicks")
)
def reset_parameters(n_clicks):
    return (
        initial_params["amplitude"],
        initial_params["frequency"],
        initial_params["phase"],
        initial_params["noise_mean"],
        initial_params["noise_covariance"],
        initial_params["filter_window_size"],
        ["show_noise"] if initial_params["show_noise"] else [],
        "both",
    )

# Run the app
app.run_server(debug=True, port=8051)


In [8]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash, dcc, html, Input, Output, State

# Function to generate harmonic with noise
def harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise, time):
    harmonic = amplitude * np.sin(2 * np.pi * frequency * time + phase)
    noise = np.random.normal(noise_mean, np.sqrt(noise_covariance), size=len(time))
    if show_noise:
        return harmonic + noise, noise
    return harmonic, None

# Custom filter function
def custom_filter(signal, window_size):
    if window_size < 1:
        raise ValueError("Window size must be >= 1")

    kernel = np.ones(window_size) / window_size
    filtered_signal = np.convolve(signal, kernel, mode='same')
    return filtered_signal

# Initial parameters
initial_params = {
    "amplitude": 1.0,
    "frequency": 1.0,
    "phase": 0.0,
    "noise_mean": 0.0,
    "noise_covariance": 0.1,
    "show_noise": True,
    "filter_window_size": 5
}

# Time range
time = np.linspace(0, 2, 1000)

# Create Dash app
app = Dash(__name__)

# User interface
app.layout = html.Div([
    html.H1("Harmonic with Noise", style={"textAlign": "center", "color": "white"}),

    dcc.Graph(id="harmonic-plot"),

    html.Div([
        html.Div([
            html.Label(id="amplitude-label", children=f"Amplitude: {initial_params['amplitude']:.1f}", style={"color": "white"}),
            dcc.Slider(id="amplitude-slider", min=0.1, max=5.0, step=0.1, value=initial_params["amplitude"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="frequency-label", children=f"Frequency: {initial_params['frequency']:.1f}", style={"color": "white"}),
            dcc.Slider(id="frequency-slider", min=0.1, max=5.0, step=0.1, value=initial_params["frequency"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="phase-label", children=f"Phase Shift: {initial_params['phase']:.1f}", style={"color": "white"}),
            dcc.Slider(id="phase-slider", min=0.0, max=2 * np.pi, step=0.1, value=initial_params["phase"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="noise-mean-label", children=f"Noise Mean: {initial_params['noise_mean']:.1f}", style={"color": "white"}),
            dcc.Slider(id="noise-mean-slider", min=-1.0, max=1.0, step=0.1, value=initial_params["noise_mean"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="noise-covariance-label", children=f"Noise Covariance: {initial_params['noise_covariance']:.2f}", style={"color": "white"}),
            dcc.Slider(id="noise-covariance-slider", min=0.01, max=1.0, step=0.01, value=initial_params["noise_covariance"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="filter-window-label", children=f"Filter Window Size: {initial_params['filter_window_size']}", style={"color": "white"}),
            dcc.Slider(id="filter-window-slider", min=1, max=50, step=1, value=initial_params["filter_window_size"]),
        ], style={"margin-bottom": "20px"}),

        html.Label("Show Noise", style={"color": "white"}),
        dcc.Checklist(id="show-noise-checkbox", options=[{"label": "Noise", "value": "show_noise"}],
                      value=["show_noise"] if initial_params["show_noise"] else [], style={"color": "white"}),

        html.Label("Select Plot", style={"color": "white"}),
        dcc.Dropdown(
            id="plot-dropdown",
            options=[
                {"label": "Harmonic Only", "value": "harmonic"},
                {"label": "Noise Only", "value": "noise"},
                {"label": "Both", "value": "both"},
                {"label": "Filtered Harmonic", "value": "filtered"},
            ],
            value="both",
        ),

        html.Button("Reset", id="reset-button", n_clicks=0, style={"margin-top": "20px", "background-color": "#4CAF50", "color": "white", "border": "none", "padding": "10px 20px", "cursor": "pointer"}),
    ], style={"width": "50%", "margin": "auto"})
], style={"background-color": "#2c3e50", "padding": "20px"})

@app.callback(
    Output("harmonic-plot", "figure"),
    Input("amplitude-slider", "value"),
    Input("frequency-slider", "value"),
    Input("phase-slider", "value"),
    Input("noise-mean-slider", "value"),
    Input("noise-covariance-slider", "value"),
    Input("filter-window-slider", "value"),
    Input("show-noise-checkbox", "value"),
    Input("plot-dropdown", "value"),
    State("harmonic-plot", "figure")
)
def update_plot(amplitude, frequency, phase, noise_mean, noise_covariance, filter_window, show_noise, plot_selection, current_figure):
    show_noise_flag = "show_noise" in show_noise
    harmonic, noise = harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise_flag, time)

    if plot_selection == "filtered":
        harmonic = custom_filter(harmonic, filter_window)

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Harmonic", "Noise"))

    # Display graphs based on dropdown selection
    if plot_selection in ["harmonic", "both", "filtered"]:
        fig.add_trace(go.Scatter(x=time, y=harmonic, mode="lines", name="Harmonic", line=dict(color="cyan")), row=1, col=1)
    else:
        # Add image in place of Harmonic graph
        fig.add_layout_image(
            dict(
                source="./ff0d15a43d0fd6168abec490b6fbdb02.jpg.png",  # Path to the image
                xref="paper", yref="paper",
                x=0, y=1,  # Position in the graph
                sizex=1, sizey=1,  # Scale of the image
                xanchor="left", yanchor="top",
                layer="below"
            )
        )

    if plot_selection in ["noise", "both"] and noise is not None:
        fig.add_trace(go.Scatter(x=time, y=noise, mode="lines", name="Noise", line=dict(color="magenta", dash="dot")), row=2, col=1)
    else:
        # Add image in place of Noise graph
        fig.add_layout_image(
            dict(
                source="./ff0d15a43d0fd6168abec490b6fbdb02.jpg.png",  # Path to the image
                xref="paper", yref="paper",
                x=0, y=0.5,  # Position in the graph
                sizex=1, sizey=0.5,  # Scale of the image
                xanchor="left", yanchor="top",
                layer="below"
            )
        )

    fig.update_layout(
        height=600,
        title="Harmonic with Noise",
        title_font=dict(color="white"),
        plot_bgcolor="#34495e",
        paper_bgcolor="#2c3e50",
        font=dict(color="white")
    )

    return fig



# Update slider labels
@app.callback(
    [Output("amplitude-label", "children"),
     Output("frequency-label", "children"),
     Output("phase-label", "children"),
     Output("noise-mean-label", "children"),
     Output("noise-covariance-label", "children"),
     Output("filter-window-label", "children")],
    [Input("amplitude-slider", "value"),
     Input("frequency-slider", "value"),
     Input("phase-slider", "value"),
     Input("noise-mean-slider", "value"),
     Input("noise-covariance-slider", "value"),
     Input("filter-window-slider", "value")]
)
def update_labels(amplitude, frequency, phase, noise_mean, noise_covariance, filter_window):
    return (
        f"Amplitude: {amplitude:.1f}",
        f"Frequency: {frequency:.1f}",
        f"Phase Shift: {phase:.1f}",
        f"Noise Mean: {noise_mean:.1f}",
        f"Noise Covariance: {noise_covariance:.2f}",
        f"Filter Window Size: {filter_window}"
    )

# Reset parameters
@app.callback(
    [Output("amplitude-slider", "value"),
     Output("frequency-slider", "value"),
     Output("phase-slider", "value"),
     Output("noise-mean-slider", "value"),
     Output("noise-covariance-slider", "value"),
     Output("filter-window-slider", "value"),
     Output("show-noise-checkbox", "value"),
     Output("plot-dropdown", "value")],
    Input("reset-button", "n_clicks")
)
def reset_parameters(n_clicks):
    return (
        initial_params["amplitude"],
        initial_params["frequency"],
        initial_params["phase"],
        initial_params["noise_mean"],
        initial_params["noise_covariance"],
        initial_params["filter_window_size"],
        ["show_noise"] if initial_params["show_noise"] else [],
        "both",
    )

# Run the app
app.run_server(debug=True, port=8051)


In [12]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash, dcc, html, Input, Output, State

# Function to generate harmonic with noise
def harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise, time):
    harmonic = amplitude * np.sin(2 * np.pi * frequency * time + phase)
    noise = np.random.normal(noise_mean, np.sqrt(noise_covariance), size=len(time))
    if show_noise:
        return harmonic + noise, noise
    return harmonic, None

# Custom filter function
def custom_filter(signal, window_size):
    if window_size < 1:
        raise ValueError("Window size must be >= 1")

    kernel = np.ones(window_size) / window_size
    filtered_signal = np.convolve(signal, kernel, mode='same')
    return filtered_signal

# Initial parameters
initial_params = {
    "amplitude": 1.0,
    "frequency": 1.0,
    "phase": 0.0,
    "noise_mean": 0.0,
    "noise_covariance": 0.1,
    "show_noise": True,
    "filter_window_size": 5
}

# Time range
time = np.linspace(0, 2, 1000)

# Create Dash app
app = Dash(__name__)

# User interface
app.layout = html.Div([
    html.H1("Harmonic with Noise", style={"textAlign": "center", "color": "white"}),

    dcc.Graph(id="harmonic-plot"),

    html.Div([
        html.Div([
            html.Label(id="amplitude-label", children=f"Amplitude: {initial_params['amplitude']:.1f}", style={"color": "white"}),
            dcc.Slider(id="amplitude-slider", min=0.1, max=5.0, step=0.1, value=initial_params["amplitude"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="frequency-label", children=f"Frequency: {initial_params['frequency']:.1f}", style={"color": "white"}),
            dcc.Slider(id="frequency-slider", min=0.1, max=5.0, step=0.1, value=initial_params["frequency"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="phase-label", children=f"Phase Shift: {initial_params['phase']:.1f}", style={"color": "white"}),
            dcc.Slider(id="phase-slider", min=0.0, max=2 * np.pi, step=0.1, value=initial_params["phase"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="noise-mean-label", children=f"Noise Mean: {initial_params['noise_mean']:.1f}", style={"color": "white"}),
            dcc.Slider(id="noise-mean-slider", min=-1.0, max=1.0, step=0.1, value=initial_params["noise_mean"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="noise-covariance-label", children=f"Noise Covariance: {initial_params['noise_covariance']:.2f}", style={"color": "white"}),
            dcc.Slider(id="noise-covariance-slider", min=0.01, max=1.0, step=0.01, value=initial_params["noise_covariance"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="filter-window-label", children=f"Filter Window Size: {initial_params['filter_window_size']}", style={"color": "white"}),
            dcc.Slider(id="filter-window-slider", min=1, max=50, step=1, value=initial_params["filter_window_size"]),
        ], style={"margin-bottom": "20px"}),

        html.Label("Show Noise", style={"color": "white"}),
        dcc.Checklist(id="show-noise-checkbox", options=[{"label": "Noise", "value": "show_noise"}],
                      value=["show_noise"] if initial_params["show_noise"] else [], style={"color": "white"}),

        html.Label("Select Plot", style={"color": "white"}),
        dcc.Dropdown(
            id="plot-dropdown",
            options=[
                {"label": "Harmonic Only", "value": "harmonic"},
                {"label": "Noise Only", "value": "noise"},
                {"label": "Both", "value": "both"},
                {"label": "Filtered Harmonic", "value": "filtered"},
            ],
            value="both",
        ),

        html.Button("Reset", id="reset-button", n_clicks=0, style={"margin-top": "20px", "background-color": "#4CAF50", "color": "white", "border": "none", "padding": "10px 20px", "cursor": "pointer"}),
    ], style={"width": "50%", "margin": "auto"})
], style={"background-color": "#2c3e50", "padding": "20px"})

# Update the plot
@app.callback(
    Output("harmonic-plot", "figure"),
    Input("amplitude-slider", "value"),
    Input("frequency-slider", "value"),
    Input("phase-slider", "value"),
    Input("noise-mean-slider", "value"),
    Input("noise-covariance-slider", "value"),
    Input("filter-window-slider", "value"),
    Input("show-noise-checkbox", "value"),
    Input("plot-dropdown", "value"),
    State("harmonic-plot", "figure")
)
def update_plot(amplitude, frequency, phase, noise_mean, noise_covariance, filter_window, show_noise, plot_selection, current_figure):
    show_noise_flag = "show_noise" in show_noise
    harmonic, noise = harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise_flag, time)

    if plot_selection == "filtered":
        harmonic = custom_filter(harmonic, filter_window)

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Harmonic", "Noise"))

    # Harmonic plot or heart
    if plot_selection in ["harmonic", "both", "filtered"]:
        fig.add_trace(go.Scatter(x=time, y=harmonic, mode="lines", name="Harmonic", line=dict(color="cyan")), row=1, col=1)
    else:
        # Draw a heart in the top graph
        t = np.linspace(0, 2 * np.pi, 500)
        x = 16 * np.sin(t)**3
        y = 13 * np.cos(t) - 5 * np.cos(2 * t) - 2 * np.cos(3 * t) - np.cos(4 * t)
        fig.add_trace(go.Scatter(x=x, y=y, mode="lines", name="Heart", line=dict(color="red")), row=1, col=1)

    # Noise plot or heart
    if plot_selection in ["noise", "both"] and noise is not None:
        fig.add_trace(go.Scatter(x=time, y=noise, mode="lines", name="Noise", line=dict(color="magenta", dash="dot")), row=2, col=1)
    else:
        # Draw a heart in the bottom graph
        t = np.linspace(0, 2 * np.pi, 500)
        x = 16 * np.sin(t)**3
        y = 13 * np.cos(t) - 5 * np.cos(2 * t) - 2 * np.cos(3 * t) - np.cos(4 * t)
        fig.add_trace(go.Scatter(x=x, y=y, mode="lines", name="Heart", line=dict(color="pink")), row=2, col=1)

    fig.update_layout(
        height=600,
        title="Harmonic with Noise or Hearts",
        title_font=dict(color="white"),
        plot_bgcolor="#34495e",
        paper_bgcolor="#2c3e50",
        font=dict(color="white")
    )

    return fig

# Update slider labels
@app.callback(
    [Output("amplitude-label", "children"),
     Output("frequency-label", "children"),
     Output("phase-label", "children"),
     Output("noise-mean-label", "children"),
     Output("noise-covariance-label", "children"),
     Output("filter-window-label", "children")],
    [Input("amplitude-slider", "value"),
     Input("frequency-slider", "value"),
     Input("phase-slider", "value"),
     Input("noise-mean-slider", "value"),
     Input("noise-covariance-slider", "value"),
     Input("filter-window-slider", "value")]
)
def update_labels(amplitude, frequency, phase, noise_mean, noise_covariance, filter_window):
    return (
        f"Amplitude: {amplitude:.1f}",
        f"Frequency: {frequency:.1f}",
        f"Phase Shift: {phase:.1f}",
        f"Noise Mean: {noise_mean:.1f}",
        f"Noise Covariance: {noise_covariance:.2f}",
        f"Filter Window Size: {filter_window}"
    )

# Reset parameters
@app.callback(
    [Output("amplitude-slider", "value"),
     Output("frequency-slider", "value"),
     Output("phase-slider", "value"),
     Output("noise-mean-slider", "value"),
     Output("noise-covariance-slider", "value"),
     Output("filter-window-slider", "value"),
     Output("show-noise-checkbox", "value"),
     Output("plot-dropdown", "value")],
    Input("reset-button", "n_clicks")
)
def reset_parameters(n_clicks):
    return (
        initial_params["amplitude"],
        initial_params["frequency"],
        initial_params["phase"],
        initial_params["noise_mean"],
        initial_params["noise_covariance"],
        initial_params["filter_window_size"],
        ["show_noise"] if initial_params["show_noise"] else [],
        "both",
    )

# Run the app
app.run_server(debug=True, port=8051)


In [16]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash, dcc, html, Input, Output, State

# Function to generate harmonic with noise
def harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise, time):
    harmonic = amplitude * np.sin(2 * np.pi * frequency * time + phase)
    noise = np.random.normal(noise_mean, np.sqrt(noise_covariance), size=len(time))
    if show_noise:
        return harmonic + noise, noise
    return harmonic, None

# Custom filter function
def custom_filter(signal, window_size):
    if window_size < 1:
        raise ValueError("Window size must be >= 1")

    kernel = np.ones(window_size) / window_size
    filtered_signal = np.convolve(signal, kernel, mode='same')
    return filtered_signal

# Initial parameters
initial_params = {
    "amplitude": 1.0,
    "frequency": 1.0,
    "phase": 0.0,
    "noise_mean": 0.0,
    "noise_covariance": 0.1,
    "show_noise": True,
    "filter_window_size": 5
}

# Time range
time = np.linspace(0, 2, 1000)

# Create Dash app
app = Dash(__name__)

# User interface
app.layout = html.Div([
    html.H1("Harmonic with Noise", style={"textAlign": "center", "color": "white"}),

    dcc.Graph(id="harmonic-plot"),

    html.Div([
        html.Div([
            html.Label(id="amplitude-label", children=f"Amplitude: {initial_params['amplitude']:.1f}", style={"color": "white"}),
            dcc.Slider(id="amplitude-slider", min=0.1, max=5.0, step=0.1, value=initial_params["amplitude"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="frequency-label", children=f"Frequency: {initial_params['frequency']:.1f}", style={"color": "white"}),
            dcc.Slider(id="frequency-slider", min=0.1, max=5.0, step=0.1, value=initial_params["frequency"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="phase-label", children=f"Phase Shift: {initial_params['phase']:.1f}", style={"color": "white"}),
            dcc.Slider(id="phase-slider", min=0.0, max=2 * np.pi, step=0.1, value=initial_params["phase"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="noise-mean-label", children=f"Noise Mean: {initial_params['noise_mean']:.1f}", style={"color": "white"}),
            dcc.Slider(id="noise-mean-slider", min=-1.0, max=1.0, step=0.1, value=initial_params["noise_mean"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="noise-covariance-label", children=f"Noise Covariance: {initial_params['noise_covariance']:.2f}", style={"color": "white"}),
            dcc.Slider(id="noise-covariance-slider", min=0.01, max=1.0, step=0.01, value=initial_params["noise_covariance"]),
        ], style={"margin-bottom": "20px"}),

        html.Div([
            html.Label(id="filter-window-label", children=f"Filter Window Size: {initial_params['filter_window_size']}", style={"color": "white"}),
            dcc.Slider(id="filter-window-slider", min=1, max=50, step=1, value=initial_params["filter_window_size"]),
        ], style={"margin-bottom": "20px"}),

        html.Label("Show Noise", style={"color": "white"}),
        dcc.Checklist(id="show-noise-checkbox", options=[{"label": "Noise", "value": "show_noise"}],
                      value=["show_noise"] if initial_params["show_noise"] else [], style={"color": "white"}),

        html.Label("Select Plot", style={"color": "white"}),
        dcc.Dropdown(
            id="plot-dropdown",
            options=[
                {"label": "Harmonic Only", "value": "harmonic"},
                {"label": "Noise Only", "value": "noise"},
                {"label": "Both", "value": "both"},
                {"label": "Filtered Harmonic", "value": "filtered"},
            ],
            value="both",
        ),

        html.Button("Reset", id="reset-button", n_clicks=0, style={"margin-top": "20px", "background-color": "#4CAF50", "color": "white", "border": "none", "padding": "10px 20px", "cursor": "pointer"}),
    ], style={"width": "50%", "margin": "auto"})
], style={"background-color": "#2c3e50", "padding": "20px"})


@app.callback(
    Output("harmonic-plot", "figure"),
    Input("amplitude-slider", "value"),
    Input("frequency-slider", "value"),
    Input("phase-slider", "value"),
    Input("noise-mean-slider", "value"),
    Input("noise-covariance-slider", "value"),
    Input("filter-window-slider", "value"),
    Input("show-noise-checkbox", "value"),
    Input("plot-dropdown", "value"),
    State("harmonic-plot", "figure")
)
def update_plot(amplitude, frequency, phase, noise_mean, noise_covariance, filter_window, show_noise, plot_selection, current_figure):
    show_noise_flag = "show_noise" in show_noise
    harmonic, noise = harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise_flag, time)

    # Форма сердечка для замены пустого графика
    t = np.linspace(0, 2 * np.pi, 500)
    x = 16 * np.sin(t)**3 * 0.1  # Уменьшение масштаба по x
    y = (13 * np.cos(t) - 5 * np.cos(2 * t) - 2 * np.cos(3 * t) - np.cos(4 * t)) * 0.1  # Уменьшение масштаба по y

    # Динамическое создание графиков в зависимости от выбора
    if plot_selection in ["harmonic", "filtered"]:  # Только верхний график
        fig = make_subplots(rows=1, cols=1, subplot_titles=("Harmonic",))
        if plot_selection == "filtered":
            harmonic = custom_filter(harmonic, filter_window)
        fig.add_trace(go.Scatter(x=time, y=harmonic, mode="lines", name="Harmonic", line=dict(color="cyan")), row=1, col=1)
    elif plot_selection == "noise":  # Только нижний график
        fig = make_subplots(rows=1, cols=1, subplot_titles=("Noise",))
        fig.add_trace(go.Scatter(x=time, y=noise, mode="lines", name="Noise", line=dict(color="magenta", dash="dot")), row=1, col=1)
    elif plot_selection == "both":  # Оба графика
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Harmonic", "Noise"))
        fig.add_trace(go.Scatter(x=time, y=harmonic, mode="lines", name="Harmonic", line=dict(color="cyan")), row=1, col=1)
        fig.add_trace(go.Scatter(x=time, y=noise, mode="lines", name="Noise", line=dict(color="magenta", dash="dot")), row=2, col=1)
    else:  # Когда нет данных
        fig = make_subplots(rows=1, cols=1, subplot_titles=("Heart",))
        fig.add_trace(go.Scatter(x=x, y=y, mode="lines", name="Heart", line=dict(color="red")), row=1, col=1)

    # Установка общего оформления
    fig.update_layout(
        height=600,
        title="Harmonic with Noise or Hearts",
        title_font=dict(color="white"),
        plot_bgcolor="#34495e",
        paper_bgcolor="#2c3e50",
        font=dict(color="white")
    )

    # Настройка осей, чтобы сердечко не занимало всё пространство
    if plot_selection not in ["harmonic", "noise", "both", "filtered"]:
        fig.update_xaxes(range=[-2, 2], title="")  # Уменьшаем диапазон по X
        fig.update_yaxes(range=[-2, 2], title="")  # Уменьшаем диапазон по Y

    return fig



# Update slider labels
@app.callback(
    [Output("amplitude-label", "children"),
     Output("frequency-label", "children"),
     Output("phase-label", "children"),
     Output("noise-mean-label", "children"),
     Output("noise-covariance-label", "children"),
     Output("filter-window-label", "children")],
    [Input("amplitude-slider", "value"),
     Input("frequency-slider", "value"),
     Input("phase-slider", "value"),
     Input("noise-mean-slider", "value"),
     Input("noise-covariance-slider", "value"),
     Input("filter-window-slider", "value")]
)
def update_labels(amplitude, frequency, phase, noise_mean, noise_covariance, filter_window):
    return (
        f"Amplitude: {amplitude:.1f}",
        f"Frequency: {frequency:.1f}",
        f"Phase Shift: {phase:.1f}",
        f"Noise Mean: {noise_mean:.1f}",
        f"Noise Covariance: {noise_covariance:.2f}",
        f"Filter Window Size: {filter_window}"
    )

# Reset parameters
@app.callback(
    [Output("amplitude-slider", "value"),
     Output("frequency-slider", "value"),
     Output("phase-slider", "value"),
     Output("noise-mean-slider", "value"),
     Output("noise-covariance-slider", "value"),
     Output("filter-window-slider", "value"),
     Output("show-noise-checkbox", "value"),
     Output("plot-dropdown", "value")],
    Input("reset-button", "n_clicks")
)
def reset_parameters(n_clicks):
    return (
        initial_params["amplitude"],
        initial_params["frequency"],
        initial_params["phase"],
        initial_params["noise_mean"],
        initial_params["noise_covariance"],
        initial_params["filter_window_size"],
        ["show_noise"] if initial_params["show_noise"] else [],
        "both",
    )

# Run the app
app.run_server(debug=True, port=8051)
